In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
from enum import IntEnum
import sys
sys.path.append("./utils")

from generate_features import *
from generate_features2 import *
from get_targets import *
from dnn_utils import *

from stockstats import StockDataFrame

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, fbeta_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns

import optuna

Current device: cuda


In [2]:
# def plot_scores(scores, title):
#     plt.figure(figsize=(15, 6))
#     labels = ['Без эмбэддингов', 'Core Web Sm', 'L&Mc', 
#               'Affective Space', 'Affective Space + индикаторы', 'Roberta Base + индикаторы']
#     plt.title(title)
#     sns.barplot(labels, scores)
#     plt.xticks(rotation=45)

In [19]:
# best_params = {'time_lag': 48,
#  'window': 17,
#  'factor': 1.0051792483424578,
#  'ewm_periods': 19,
#  'ewm_alpha': 0.49161110885419984
# }

best_params = {'time_lag': 38,
 'window': 25,
 'factor': 5.230229226959859,
 'ewm_periods': 11,
 'ewm_alpha': 0.25825439118706905
}

# best_params = {
#     'time_lag': 46,
#     'window': 30,
#     'factor': 14.825366367433316,
#     'ewm_periods': 19,
#     'ewm_alpha': 0.19576050683568483
# }

In [20]:
TEST_RATIO = 0.2
EWM_PERIODS = best_params["ewm_periods"]
EWM_ALPHA = best_params["ewm_alpha"]

TIME_LAG = best_params["time_lag"]
WINDOW = best_params["window"]
FACTOR = best_params["factor"]

In [21]:
# ROBERTA_BASE_EMBEDDING_PATH = 'roberta_base_embeddings.csv'
# CORE_WEB_SM_PATH = 'core_web_sm_embeddings.csv'
# AFS_RESULT_PATH = "tesla_dataset_affectivespace.csv"
# LMC_RESULT_PATH = "tesla_dataset_l&mc.csv"

AFS_RESULT_BTC_PATH = 'data_crypto_2021/afs_all_btc_embeddings.csv'
RB_RESULT_BTC_PATH = 'data_crypto_2021/rb_all_btc_embeddings.csv'
ROBERTA_BASE_EMBEDDING_PATH = 'btc_roberta_base.csv'


train_df = pd.read_csv(RB_RESULT_BTC_PATH, index_col=0)

In [22]:
train_df = indicators_with_rowwise_ewm(train_df)
train_df

,open,high,low,close,volume,0,1,2,3,4,...,chop_86_ewm,chop_174_ewm,chop_363_ewm,mfi_6_ewm,mfi_14_ewm,mfi_26_ewm,mfi_42_ewm,mfi_86_ewm,mfi_174_ewm,mfi_363_ewm
0,29484.39,29535.95,29484.39,29518.58,420109.235827,0.455301,0.357461,0.083388,0.075250,0.021029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29459.58,29459.58,29429.67,29439.16,356581.967102,0.455301,0.357461,0.083388,0.075250,0.021029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29446.25,29450.00,29439.07,29441.25,51637.618692,0.455301,0.357461,0.083388,0.075250,0.021029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29353.67,29396.07,29338.14,29370.50,170835.156823,0.455301,0.357461,0.083388,0.075250,0.021029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29398.82,29418.94,29375.71,29377.43,191779.310775,0.164001,-0.014154,-0.496314,0.138247,0.306280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106043,46468.80,46468.80,46420.09,46420.09,135536.969821,-0.047028,-0.101962,0.003670,-0.376551,0.309242,...,27.553482,31.100160,40.072669,0.401612,0.450092,0.644193,0.515252,0.240690,0.492479,0.442104
106044,46389.97,46420.87,46385.63,46420.87,11566.314954,-0.047028,-0.101962,0.003670,-0.376551,0.309242,...,27.924360,30.954001,40.058480,0.466274,0.474871,0.645452,0.546870,0.243479,0.498231,0.443987
106045,46434.34,46434.34,46421.01,46428.48,7935.098945,-0.047028,-0.101962,0.003670,-0.376551,0.309242,...,28.204027,30.817515,40.033494,0.515393,0.495141,0.623696,0.573669,0.246120,0.502700,0.445351
106046,46368.00,46368.00,46354.76,46356.31,96443.304904,-0.047028,-0.101962,0.003670,-0.376551,0.309242,...,28.423178,30.706781,40.008292,0.528768,0.496367,0.581564,0.578256,0.246953,0.505013,0.445807


In [23]:
targets = get_buy_signal_with_std_threshold_target(train_df, TIME_LAG, WINDOW, FACTOR)
train_df = pd.concat([train_df, targets], axis=1)
train_df.dropna(inplace=True)

train_df.reset_index(drop=True, inplace=True)

# train_df.drop(columns=[str(i) for i in range(100)], inplace=True)

In [90]:
train_df.drop(columns=unimportant_features, inplace=True)

In [24]:
X = train_df.drop(["target"], axis=1).to_numpy()
y = train_df["target"].to_numpy()

X.shape, y.shape

((105972, 1704), (105972,))

In [25]:
train_df['target'].value_counts()

0.0    96159
1.0     9813
Name: target, dtype: int64

In [26]:
test_size = int(X.shape[0] * TEST_RATIO)

X_train = X[:-test_size]
y_train = y[:-test_size]
X_test = X[-test_size:]
y_test = y[-test_size:]

train_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
test_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_test), y=y_test)

In [55]:
compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train) * np.array([5, 1])

array([4.24435721, 1.21659634])

In [27]:
lgbm_classifier = LGBMClassifier(class_weight="balanced")
lgbm_classifier.fit(X_train, y_train)

LGBMClassifier(class_weight='balanced')

**Affective Space**

In [11]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
print("roc_auc_score:", roc_auc_score(y_test, probas[:, 1]))
print("fbeta_score:", fbeta_score(y_test, predictions, beta=0.3))
print("accuracy_score:", accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions)

roc_auc_score: 0.632246393640225
fbeta_score: 0.15001290248587898
accuracy_score: 0.855996980277437


array([[17902,  1327],
       [ 1725,   240]], dtype=int64)

**Roberta Base**

In [29]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
print("roc_auc_score:", roc_auc_score(y_test, probas[:, 1]))
print("fbeta_score:", fbeta_score(y_test, predictions, beta=0.3))
print("accuracy_score:", accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions)

roc_auc_score: 0.6261066002805082
fbeta_score: 0.1318063804815592
accuracy_score: 0.8653392469566858


array([[18170,  1059],
       [ 1795,   170]], dtype=int64)

In [30]:
cb_classifier = CatBoostClassifier(class_weights=train_weights)
cb_classifier.fit(X_train, y_train)

Learning rate set to 0.068598
0:	learn: 0.6837827	total: 409ms	remaining: 6m 48s
1:	learn: 0.6751824	total: 742ms	remaining: 6m 10s
2:	learn: 0.6669362	total: 1.1s	remaining: 6m 5s
3:	learn: 0.6593429	total: 1.46s	remaining: 6m 2s
4:	learn: 0.6534384	total: 1.82s	remaining: 6m 2s
5:	learn: 0.6474316	total: 2.2s	remaining: 6m 4s
6:	learn: 0.6419272	total: 2.53s	remaining: 5m 58s
7:	learn: 0.6369671	total: 2.86s	remaining: 5m 55s
8:	learn: 0.6322716	total: 3.18s	remaining: 5m 49s
9:	learn: 0.6274266	total: 3.53s	remaining: 5m 49s
10:	learn: 0.6235296	total: 3.87s	remaining: 5m 48s
11:	learn: 0.6194773	total: 4.25s	remaining: 5m 50s
12:	learn: 0.6155665	total: 4.58s	remaining: 5m 48s
13:	learn: 0.6123580	total: 4.88s	remaining: 5m 43s
14:	learn: 0.6086524	total: 5.23s	remaining: 5m 43s
15:	learn: 0.6051584	total: 5.52s	remaining: 5m 39s
16:	learn: 0.6014545	total: 5.86s	remaining: 5m 38s
17:	learn: 0.5977771	total: 6.2s	remaining: 5m 38s
18:	learn: 0.5945231	total: 6.57s	remaining: 5m 39s

158:	learn: 0.3743448	total: 53.6s	remaining: 4m 43s
159:	learn: 0.3733889	total: 54s	remaining: 4m 43s
160:	learn: 0.3727188	total: 54.4s	remaining: 4m 43s
161:	learn: 0.3719961	total: 54.8s	remaining: 4m 43s
162:	learn: 0.3712820	total: 55.1s	remaining: 4m 43s
163:	learn: 0.3703653	total: 55.6s	remaining: 4m 43s
164:	learn: 0.3695153	total: 55.9s	remaining: 4m 43s
165:	learn: 0.3684435	total: 56.3s	remaining: 4m 43s
166:	learn: 0.3673684	total: 56.8s	remaining: 4m 43s
167:	learn: 0.3666175	total: 57.2s	remaining: 4m 43s
168:	learn: 0.3659349	total: 57.5s	remaining: 4m 42s
169:	learn: 0.3647704	total: 57.9s	remaining: 4m 42s
170:	learn: 0.3637448	total: 58.4s	remaining: 4m 42s
171:	learn: 0.3627066	total: 58.8s	remaining: 4m 43s
172:	learn: 0.3615334	total: 59.2s	remaining: 4m 42s
173:	learn: 0.3603019	total: 59.6s	remaining: 4m 42s
174:	learn: 0.3586780	total: 1m	remaining: 4m 42s
175:	learn: 0.3575413	total: 1m	remaining: 4m 42s
176:	learn: 0.3566334	total: 1m	remaining: 4m 42s
177:

311:	learn: 0.2496066	total: 1m 57s	remaining: 4m 18s
312:	learn: 0.2491206	total: 1m 57s	remaining: 4m 17s
313:	learn: 0.2484202	total: 1m 57s	remaining: 4m 17s
314:	learn: 0.2479032	total: 1m 58s	remaining: 4m 17s
315:	learn: 0.2472879	total: 1m 58s	remaining: 4m 17s
316:	learn: 0.2468218	total: 1m 59s	remaining: 4m 16s
317:	learn: 0.2461589	total: 1m 59s	remaining: 4m 16s
318:	learn: 0.2456961	total: 2m	remaining: 4m 16s
319:	learn: 0.2450349	total: 2m	remaining: 4m 15s
320:	learn: 0.2443852	total: 2m	remaining: 4m 15s
321:	learn: 0.2438423	total: 2m 1s	remaining: 4m 15s
322:	learn: 0.2433959	total: 2m 1s	remaining: 4m 15s
323:	learn: 0.2429666	total: 2m 2s	remaining: 4m 14s
324:	learn: 0.2424092	total: 2m 2s	remaining: 4m 14s
325:	learn: 0.2418560	total: 2m 3s	remaining: 4m 14s
326:	learn: 0.2411705	total: 2m 3s	remaining: 4m 14s
327:	learn: 0.2406649	total: 2m 3s	remaining: 4m 13s
328:	learn: 0.2401940	total: 2m 4s	remaining: 4m 13s
329:	learn: 0.2396213	total: 2m 4s	remaining: 4m

465:	learn: 0.1832195	total: 3m	remaining: 3m 27s
466:	learn: 0.1828297	total: 3m 1s	remaining: 3m 26s
467:	learn: 0.1825672	total: 3m 1s	remaining: 3m 26s
468:	learn: 0.1821836	total: 3m 1s	remaining: 3m 26s
469:	learn: 0.1818392	total: 3m 2s	remaining: 3m 25s
470:	learn: 0.1815033	total: 3m 2s	remaining: 3m 25s
471:	learn: 0.1812359	total: 3m 3s	remaining: 3m 25s
472:	learn: 0.1809214	total: 3m 3s	remaining: 3m 24s
473:	learn: 0.1806764	total: 3m 4s	remaining: 3m 24s
474:	learn: 0.1803479	total: 3m 4s	remaining: 3m 23s
475:	learn: 0.1800419	total: 3m 4s	remaining: 3m 23s
476:	learn: 0.1797815	total: 3m 5s	remaining: 3m 23s
477:	learn: 0.1795434	total: 3m 5s	remaining: 3m 22s
478:	learn: 0.1791266	total: 3m 6s	remaining: 3m 22s
479:	learn: 0.1787528	total: 3m 6s	remaining: 3m 22s
480:	learn: 0.1784331	total: 3m 6s	remaining: 3m 21s
481:	learn: 0.1780740	total: 3m 7s	remaining: 3m 21s
482:	learn: 0.1777805	total: 3m 7s	remaining: 3m 21s
483:	learn: 0.1774122	total: 3m 8s	remaining: 3m 

619:	learn: 0.1427895	total: 4m 6s	remaining: 2m 31s
620:	learn: 0.1425065	total: 4m 7s	remaining: 2m 30s
621:	learn: 0.1423257	total: 4m 7s	remaining: 2m 30s
622:	learn: 0.1420202	total: 4m 8s	remaining: 2m 30s
623:	learn: 0.1418288	total: 4m 8s	remaining: 2m 29s
624:	learn: 0.1417081	total: 4m 8s	remaining: 2m 29s
625:	learn: 0.1415775	total: 4m 9s	remaining: 2m 28s
626:	learn: 0.1413853	total: 4m 9s	remaining: 2m 28s
627:	learn: 0.1411450	total: 4m 10s	remaining: 2m 28s
628:	learn: 0.1409693	total: 4m 10s	remaining: 2m 27s
629:	learn: 0.1408162	total: 4m 11s	remaining: 2m 27s
630:	learn: 0.1406021	total: 4m 11s	remaining: 2m 27s
631:	learn: 0.1403223	total: 4m 12s	remaining: 2m 26s
632:	learn: 0.1401010	total: 4m 12s	remaining: 2m 26s
633:	learn: 0.1399206	total: 4m 12s	remaining: 2m 26s
634:	learn: 0.1396981	total: 4m 13s	remaining: 2m 25s
635:	learn: 0.1395572	total: 4m 13s	remaining: 2m 25s
636:	learn: 0.1393904	total: 4m 14s	remaining: 2m 24s
637:	learn: 0.1392296	total: 4m 14s	

772:	learn: 0.1158502	total: 5m 15s	remaining: 1m 32s
773:	learn: 0.1156950	total: 5m 16s	remaining: 1m 32s
774:	learn: 0.1155757	total: 5m 16s	remaining: 1m 31s
775:	learn: 0.1153878	total: 5m 17s	remaining: 1m 31s
776:	learn: 0.1152461	total: 5m 17s	remaining: 1m 31s
777:	learn: 0.1151222	total: 5m 18s	remaining: 1m 30s
778:	learn: 0.1149657	total: 5m 18s	remaining: 1m 30s
779:	learn: 0.1147999	total: 5m 19s	remaining: 1m 29s
780:	learn: 0.1147048	total: 5m 19s	remaining: 1m 29s
781:	learn: 0.1145495	total: 5m 19s	remaining: 1m 29s
782:	learn: 0.1144091	total: 5m 20s	remaining: 1m 28s
783:	learn: 0.1142094	total: 5m 20s	remaining: 1m 28s
784:	learn: 0.1140409	total: 5m 21s	remaining: 1m 27s
785:	learn: 0.1139273	total: 5m 21s	remaining: 1m 27s
786:	learn: 0.1137914	total: 5m 22s	remaining: 1m 27s
787:	learn: 0.1136565	total: 5m 22s	remaining: 1m 26s
788:	learn: 0.1135207	total: 5m 23s	remaining: 1m 26s
789:	learn: 0.1133723	total: 5m 23s	remaining: 1m 26s
790:	learn: 0.1132287	total:

927:	learn: 0.0964163	total: 6m 20s	remaining: 29.5s
928:	learn: 0.0963102	total: 6m 21s	remaining: 29.1s
929:	learn: 0.0962039	total: 6m 21s	remaining: 28.7s
930:	learn: 0.0961083	total: 6m 21s	remaining: 28.3s
931:	learn: 0.0960401	total: 6m 22s	remaining: 27.9s
932:	learn: 0.0959604	total: 6m 22s	remaining: 27.5s
933:	learn: 0.0957807	total: 6m 23s	remaining: 27.1s
934:	learn: 0.0957036	total: 6m 23s	remaining: 26.7s
935:	learn: 0.0955857	total: 6m 24s	remaining: 26.3s
936:	learn: 0.0955052	total: 6m 24s	remaining: 25.9s
937:	learn: 0.0954237	total: 6m 24s	remaining: 25.4s
938:	learn: 0.0953508	total: 6m 25s	remaining: 25s
939:	learn: 0.0952500	total: 6m 25s	remaining: 24.6s
940:	learn: 0.0951815	total: 6m 26s	remaining: 24.2s
941:	learn: 0.0950971	total: 6m 26s	remaining: 23.8s
942:	learn: 0.0950087	total: 6m 27s	remaining: 23.4s
943:	learn: 0.0949191	total: 6m 27s	remaining: 23s
944:	learn: 0.0948448	total: 6m 27s	remaining: 22.6s
945:	learn: 0.0947469	total: 6m 28s	remaining: 22.

**Affective Space**

In [18]:
probas = cb_classifier.predict_proba(X_test)
predictions = cb_classifier.predict(X_test)
print("roc_auc_score:", roc_auc_score(y_test, probas[:, 1]))
print("fbeta_score:", fbeta_score(y_test, predictions, beta=0.3))
print("accuracy_score:", accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions)

roc_auc_score: 0.6026897456754317
fbeta_score: 0.1186607784738626
accuracy_score: 0.8976597150136831


array([[18972,   257],
       [ 1912,    53]], dtype=int64)

**Roberta Base**

In [31]:
probas = cb_classifier.predict_proba(X_test)
predictions = cb_classifier.predict(X_test)
print("roc_auc_score:", roc_auc_score(y_test, probas[:, 1]))
print("fbeta_score:", fbeta_score(y_test, predictions, beta=0.3))
print("accuracy_score:", accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions)

roc_auc_score: 0.616741756017635
fbeta_score: 0.07805751370706054
accuracy_score: 0.8975653486835897


array([[18991,   238],
       [ 1933,    32]], dtype=int64)

In [30]:
feature_importance = [(imp, feat) for imp, feat in zip(cb_classifier.get_feature_importance(), train_df.columns)]
feature_importance.sort(reverse=True)
feature_importance

[(1.9928443112695864, 'volume_363_mstd_ewm'),
 (1.986870790454089, 'chop_363_ewm'),
 (1.970454383870603, 'volume_363_mstd'),
 (1.8438546142089767, 'chop_363'),
 (1.5967668700244715, 'vr_363_ewm'),
 (1.457008754581723, 'vr_174_ewm'),
 (1.4326842673982525, 'vr_6_ewm'),
 (1.409783792039762, 'volume_174_mstd'),
 (1.235618570023706, 'low_363_mstd'),
 (1.2103899293987075, 'cr-ma3_ewm'),
 (1.1726450212069885, 'mfi_363_ewm'),
 (1.143322524206914, 'volume_86_mstd'),
 (1.0679587107518542, 'chop_174_ewm'),
 (1.0220911777794934, 'volume_174_ema_ewm'),
 (0.9927025892338154, 'low_363_mstd_ewm'),
 (0.9329608097044642, 'mfi_174_ewm'),
 (0.9195921271862496, 'vr_363'),
 (0.9152574346371124, 'volume_174_mstd_ewm'),
 (0.8450298624220104, 'high_363_mstd'),
 (0.8229842313856802, 'volume_363_ema_ewm'),
 (0.7573643951908862, 'vr_174'),
 (0.7515243199195545, 'high_363_mstd_ewm'),
 (0.7428844599688769, 'volume_42_mstd'),
 (0.7313435819852878, '5'),
 (0.7295076528820005, 'boll_lb_ewm'),
 (0.7185748704077657, 'vo

In [34]:
unimportant_features = list()
for importance, feature in feature_importance:
    if importance < 0.1:
        unimportant_features += [feature]
unimportant_features

['chop_26_ewm',
 'low_42_mstd',
 'high_delta_ewm',
 '38_ewm',
 '57',
 '51_ewm',
 '35_ewm',
 'mfi_26',
 '51',
 '82_ewm',
 '57_ewm',
 'atr_3_ewm',
 '34',
 '86_ewm',
 '67_ewm',
 '58_ewm',
 'vr_14',
 '37_ewm',
 '96',
 '46_ewm',
 'pdm_14_ema',
 '11_ewm',
 'cr-ma1',
 '55_ewm',
 '53',
 'vr_14_ewm',
 'mfi_6_ewm',
 'cr-ma1_ewm',
 'volume_14_mstd_ewm',
 'adx_ewm',
 'volume_42_ema',
 '62_ewm',
 'kdjk_6',
 'change',
 'pdi_ewm',
 '78_ewm',
 'volume_6_mstd_ewm',
 'adxr',
 'mdi_14_ewm',
 'mdi',
 'wt1_ewm',
 'chop_26',
 'mfi_14',
 'mfi_14_ewm',
 'pdi_14',
 'vr_6',
 'mdi_ewm',
 'chop_14',
 'wt2',
 'dm_ewm',
 'macdh',
 'mdm_14_ema_ewm',
 'rsv_6_ewm',
 'atr_6',
 'um_ewm',
 'pdi',
 'volume_26_ema',
 'mdm_14_ewm',
 'mdm_ewm',
 'wt2_ewm',
 'mdm',
 'atr_2_ewm',
 'atr_3',
 'mfi_6',
 'chop_6_ewm',
 'kdjk_6_ewm',
 'dx_14',
 'low_delta',
 'dm',
 'high_delta',
 'volume_ewm',
 'volume_6_mstd',
 'volume',
 'um',
 'pdm_ewm',
 'pdm',
 'mdm_14_ema',
 'mdm_14',
 'dx_ewm',
 'dx_14_ewm',
 'dx',
 'chop_6',
 'atr_2',
 'adx

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**BTC**

**Affective Space**

`Было`

In [13]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_AFS_btc = roc_auc_score(y_test, probas[:, 1])
accuracy_score_AFS_btc = accuracy_score(y_test, predictions)
print(roc_auc_score_AFS_btc)
print(accuracy_score_AFS_btc)
confusion_matrix(y_test, predictions)

0.6611467236467237
0.9684241019698725


array([[3343,    5],
       [ 104,    0]], dtype=int64)

`С новыми индикаторами`

In [26]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_AFS_btc_2 = roc_auc_score(y_test, probas[:, 1])
accuracy_score_AFS_btc_2 = accuracy_score(y_test, predictions)
print(roc_auc_score_AFS_btc_2)
print(accuracy_score_AFS_btc_2)
confusion_matrix(y_test, predictions)

0.6281446355216529
0.9738673670387393


array([[7405,   16],
       [ 183,   11]], dtype=int64)

**Roberta Base**

`Было`

In [ ]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_RB_btc = roc_auc_score(y_test, probas[:, 1])
accuracy_score_RB_btc = accuracy_score(y_test, predictions)
print(roc_auc_score_RB_btc)
print(accuracy_score_RB_btc)
confusion_matrix(y_test, predictions)

`С новыми индикаторами`

In [10]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_RB_btc_2 = roc_auc_score(y_test, probas[:, 1])
accuracy_score_RB_btc_2 = accuracy_score(y_test, predictions)
print(roc_auc_score_RB_btc_2)
print(accuracy_score_RB_btc_2)
confusion_matrix(y_test, predictions)

0.6281446355216529
0.9738673670387393


array([[7405,   16],
       [ 183,   11]], dtype=int64)